In [1]:
import math
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression,LinearRegression,BayesianRidge, Lasso
from statistics import mean
from math import sqrt
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Bidirectional
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import datetime
import warnings
from tqdm import tqdm
from pathlib import Path
import time
from copy import deepcopy
import os

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [3]:
def read_data():
    calendar = pd.read_csv('calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv('sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv('sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission

In [4]:
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    # get only a sample for fst training
    data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass

    
    return data

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None 

calendar, sell_prices, sales_train_validation, submission = read_data()

data = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 28000000, merge = True)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Melted sales train validation has 58327370 rows and 8 columns
Mem. usage decreased to 3226.27 Mb (9.4% reduction)
Our final dataset to train has 31181090 rows and 18 columns


In [6]:
from sklearn.preprocessing import LabelEncoder

def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
        
    data = reduce_mem_usage(data)
    return data

In [7]:
def lag_features(data):
    
    data_fe = data[['id', 'demand']]
    
    window = 28
    periods = [7, 14, 28, 84]
    group = data_fe.groupby('id')['demand']
    
    # most recent lag data
    for period in periods:
        data_fe['demand_rolling_mean_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).mean())
        data_fe['demand_rolling_std_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).std())
        
    # reduce memory
    data_fe = reduce_mem_usage(data_fe)
    
    # get time features
    data['date'] = pd.to_datetime(data['date'])
    time_features = ['year', 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear']
    dtype = np.int16
    for time_feature in time_features:
        data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
        
    lag_rolling_features = [col for col in data_fe.columns if col not in ['id', 'demand']]
    data = pd.concat([data, data_fe[lag_rolling_features]], axis = 1)
    
    del data_fe

    return data

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31181090 entries, 0 to 31181089
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   item_id       object 
 2   dept_id       object 
 3   cat_id        object 
 4   store_id      object 
 5   state_id      object 
 6   demand        int64  
 7   part          object 
 8   date          object 
 9   wm_yr_wk      int16  
 10  event_name_1  object 
 11  event_type_1  object 
 12  event_name_2  object 
 13  event_type_2  object 
 14  snap_CA       int8   
 15  snap_TX       int8   
 16  snap_WI       int8   
 17  sell_price    float16
dtypes: float16(1), int16(1), int64(1), int8(3), object(12)
memory usage: 3.5+ GB


In [27]:
import sklearn.metrics as metrics 
from datetime import datetime, timedelta
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24']
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25']

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.09,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        'lambda_l1': 0.4,
        'lambda_l2': 0.4}
    
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
    # GroupKFold by week, month to avoid leakage and overfitting 
    kf = GroupKFold(6)
    
    # get subgroups for each week, year pair
    group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
        print(f'Training fold {fold + 1}')
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                                categorical_feature = cat_features)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                              categorical_feature = cat_features)
        
        # train with custom loss function and evaluation metric
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                          valid_sets = [train_set, val_set], verbose_eval = 20, fobj = custom_asymmetric_train, 
                          feval = custom_asymmetric_valid)
    
        model.save_model("LightGBM_CV.lgb")
        
        # predict oof
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

        # predict test
        preds += model.predict(test[features]) / 6
        
        print('-'*50)
        print('\n')
        
    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [10]:
def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('May3rdsub.csv', index = False)

In [11]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_478_CA_4_validation,HOUSEHOLD_1_478,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,NaN,NaN,NaN,NaN,1,0,0,2.669922
1,HOUSEHOLD_1_479_CA_4_validation,HOUSEHOLD_1_479,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,NaN,NaN,NaN,NaN,1,0,0,7.519531
2,HOUSEHOLD_1_480_CA_4_validation,HOUSEHOLD_1_480,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,NaN,NaN,NaN,NaN,1,0,0,NaN
3,HOUSEHOLD_1_481_CA_4_validation,HOUSEHOLD_1_481,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,NaN,NaN,NaN,NaN,1,0,0,4.968750
4,HOUSEHOLD_1_482_CA_4_validation,HOUSEHOLD_1_482,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,1,train,2013-08-04,11328,NaN,NaN,NaN,NaN,1,0,0,3.380859


In [20]:
data = lag_features(data)

Mem. usage decreased to 1011.04 Mb (61.4% reduction)


In [21]:
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 
            'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
                'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 
            'demand_rolling_mean_t14', 'demand_rolling_mean_t28', 'demand_rolling_mean_t84',
                'demand_rolling_std_t7', 'demand_rolling_std_t14', 'demand_rolling_std_t28', 'demand_rolling_std_t84']
    
cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                    'event_name_2', 'event_type_2']

In [22]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,dayofweek,dayofyear,demand_rolling_mean_t7,demand_rolling_std_t7,demand_rolling_mean_t14,demand_rolling_std_t14,demand_rolling_mean_t28,demand_rolling_std_t28,demand_rolling_mean_t84,demand_rolling_std_t84
0,HOUSEHOLD_1_478_CA_4_validation,HOUSEHOLD_1_478,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,...,6,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_479_CA_4_validation,HOUSEHOLD_1_479,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,...,6,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_480_CA_4_validation,HOUSEHOLD_1_480,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,...,6,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_481_CA_4_validation,HOUSEHOLD_1_481,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,train,2013-08-04,11328,...,6,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_482_CA_4_validation,HOUSEHOLD_1_482,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,1,train,2013-08-04,11328,...,6,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
ldata = data[(data['date'] > '2013-11-23')]

In [24]:
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [26]:
ldata.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,dayofweek,dayofyear,demand_rolling_mean_t7,demand_rolling_std_t7,demand_rolling_mean_t14,demand_rolling_std_t14,demand_rolling_mean_t28,demand_rolling_std_t28,demand_rolling_mean_t84,demand_rolling_std_t84
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2,train,2013-11-24,11344,...,6,328,0.571289,0.976074,0.428467,0.755859,0.500000,0.693848,0.440430,0.646484
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2013-11-24,11344,...,6,328,0.142822,0.377930,0.071411,0.267334,0.178589,0.390137,0.214233,0.440918
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2013-11-24,11344,...,6,328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,3,train,2013-11-24,11344,...,6,328,2.857422,2.968750,2.427734,2.208984,2.177734,2.074219,1.845703,1.820312
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2013-11-24,11344,...,6,328,0.428467,0.534668,0.928711,0.997070,0.928711,1.120117,1.190430,1.197266


In [29]:
ldata = transform(ldata)

Mem. usage decreased to 1801.29 Mb (51.8% reduction)


In [31]:
ldata = reduce_mem_usage(ldata)

Mem. usage decreased to 1801.29 Mb (0.0% reduction)


In [ ]:
next_test = run_lgb(ldata, features, cat_features)
predict(next_test, submission)